In [ ]:
config = {
    'paths':{ 
       'movie_meta_data': '/content/drive/MyDrive/NLP/Movie scripts dataset/Movie meta data/movie_meta_data.csv',
       'movies_matching_scores' : '/content/drive/MyDrive/NLP/Movie scripts dataset/Movie characters/Matching evaluation and statistics/movies_mean_matching_scores.xlsx',
       },
       'task':{
           'name': 'meta_scores' # from ['meta_scores', 'script_awards']
       }
}

In [ ]:
class DataLabels:
    def __init__(self, config):
        self.config = config
    
    def get_movie_meta_scores(self):
        movie_meta_df = pd.read_csv(self.config['paths']['movie_meta_data'])
        movie_meta_dict = dict(zip(movie_meta_df['imdbid'].tolist(), 
                                   movie_meta_df['metascore'].tolist()))    
        return movie_meta_dict

    def get_movie_script_awards(self):
        with open(config['paths']['nominated_movies'], 'rb') as f:
            nominated_movies = pickle.load(f)
        nominated_movies_imdb_ids = [x.split('_')[1].split('.')[0] for x in  nominated_movies]
        mean_matching_scores_df = pd.read_excel(config['paths']['movies_matching_scores'])
        imdb_ids = mean_matching_scores_df[mean_matching_scores_df.iou_values_mean<1.]['imdb_id'].tolist() # ИСПРАВИТЬ!!! Убрать <1
        imdb_ids = [str(x).rjust(7,'0') for x in imdb_ids]
        not_nominated_movies_imdb_ids = [x for x in imdb_ids if not x in nominated_movies_imdb_ids]
        return nominated_movies_imdb_ids, not_nominated_movies_imdb_ids

    def get_labels(self):
        if self.config['task']['name']=='meta_scores':
            movie_meta_dict = self.get_movie_meta_scores()
            movie_meta_dict = dict([(key, int(value>=60)) for key, value in movie_meta_dict.items()])
            return movie_meta_dict
        elif self.config['task']['name']=='script_awards':
            nominated_movies_imdb_ids, not_nominated_movies_imdb_ids = self.get_movie_script_awards()
            scripts_award_dict = list(zip(nominated_movies_imdb_ids, [1] * len(nominated_movies_imdb_ids)))
            scripts_award_dict += list(zip(not_nominated_movies_imdb_ids, [0] * len(nominated_movies_imdb_ids)))
            scripts_award_dict = dict(scripts_award_dict)
            return scripts_award_dict


